In [1]:
import numpy as np

In [2]:
def run_simulation(nx, ny, nz, Lx, Ly, Lz, cx, cy, cz, sx, sy, sz):
    dx, dy, dz = Lx/(nx-1), Ly/(ny-1), Lz/(nz-1)
    x = np.linspace(-2500, 2500, nx)  # Centered at (0,0)
    y = np.linspace(-2500, 2500, ny)
    z = np.linspace(0, Lz, nz)

    dt = 1
    tend = 1200
    t = 0

    cfl_x, cfl_y, cfl_z = cx * dt/dx, cy * dt/dy, cz * dt/dz
    diff_x, diff_y, diff_z = sx * dt/dx**2, sy * dt/dy**2, sz * dt/dz**2

    u = np.zeros((nx+2, ny+2, nz+2))
    sol = [u.copy()]

    source_x, source_y, source_z = nx//2, ny//2, nz//5
    Q = 1e-6

    while t < tend:
        un = sol[-1]
        unew = un.copy()

         # Advection (Upwind Scheme)
        unew[1:-1, 1:-1, 1:-1] -= cfl_x * (un[1:-1, 1:-1, 1:-1] - un[1:-1, :-2, 1:-1])
        unew[1:-1, 1:-1, 1:-1] -= cfl_y * (un[1:-1, 1:-1, 1:-1] - un[:-2, 1:-1, 1:-1])
        unew[1:-1, 1:-1, 1:-1] -= cfl_z * (un[1:-1, 1:-1, 1:-1] - un[1:-1, 1:-1, :-2])
    
        # Diffusion (Central Differencing)
        unew[1:-1, 1:-1, 1:-1] += diff_x * (un[1:-1, 2:, 1:-1] - 2*un[1:-1, 1:-1, 1:-1] + un[1:-1, :-2, 1:-1])
        unew[1:-1, 1:-1, 1:-1] += diff_y * (un[2:, 1:-1, 1:-1] - 2*un[1:-1, 1:-1, 1:-1] + un[:-2, 1:-1, 1:-1])
        unew[1:-1, 1:-1, 1:-1] += diff_z * (un[1:-1, 1:-1, 2:] - 2*un[1:-1, 1:-1, 1:-1] + un[1:-1, 1:-1, :-2])

        # Source Term
        unew[source_x, source_y, source_z] += Q * dt

        # Additional Source Points (forming a small area)
        offsets = [(-1, -1), (-1, 1), (1, -1), (1, 1), (-1, 0), (1, 0), (0, -1), (0, 1)]
        for dx, dy in offsets:
            unew[source_x + dx, source_y + dy, source_z] += Q * dt

        sol.append(unew.copy())
        t += dt

    numerical_C = sol[-1][1:-1, 1:-1, 1:-1]  # Extract final time step
    
    return x, y, z, numerical_C

In [7]:
nx, ny, nz = 51, 51, 51  # Grid points
Lx, Ly, Lz = 5000, 5000, 500  # Domain size in meters
cx, cy, cz = 0, 5, 1
sx, sy, sz = 2, 2, 0.5
x, y, z, C = run_simulation(nx, ny, nz, Lx, Ly, Lz, cx, cy, cz, sx, sy, sz)

In [4]:
observed = np.load("test.npy")

In [36]:
observed.shape, C.shape

((51, 51, 1200), (51, 51, 51))

In [34]:
temp = np.array([[[0, 1, 2], [3, 4, 5]], [[6, 7, 8], [9, 10, 11]], [[12, 13, 14], [15, 16, 17]]])
temp.shape

(3, 2, 3)